# Peak analysis Bert2GPT
run name: full_PLAbDab_healthy_human_nucleus_bert2gpt_nucleus_healthy_human_PLAbDab_max_new_tokens_115_num_epochs_50_bert_like_tokenizer_unpaired_epo_41-7  
julia version: 1.10.2

In [ ]:
using Pkg
# Pkg.add("Plots")
# Pkg.add("StatsPlots")
Pkg.add("BioSequences")
Pkg.add("BioAlignments")
Pkg.add("PyCall")

In [ ]:
using CSV
using DataFrames
using Plots
using StatsPlots
using Statistics
using BioSequences
using BioAlignments

In [ ]:
file_path = "/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2gpt/data/global_align.csv"

df = CSV.read(file_path, DataFrame);
names(df)

In [ ]:
# count unqiue values in the column generated_sequence_light
unique_values = unique(df[!, :generated_sequence_light])
length(unique_values)

# group by the column generated_sequence_light
grouped = groupby(df, :generated_sequence_light)
grouped


In [ ]:
# count unqiue values in the column generated_sequence_light
unique_values = unique(df[!, :sequence_alignment_aa_light])
length(unique_values)

In [ ]:
@df df histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=false
)

In [ ]:
using DataFrames, PyCall

# Import Python modules
sns = pyimport("seaborn")
plt = pyimport("matplotlib.pyplot")
pd = pyimport("pandas")

# Assume your Julia DataFrame is named `df`
py_df = pd.DataFrame(Dict(col => df[!, col] for col in names(df)))

In [ ]:
# Start with a fresh figure to avoid overlapping previous plots
plt.figure()

bins = 40
sns.histplot(data=py_df, x="calculated_similarity", bins=bins, kde=true, 
             color="lightskyblue", edgecolor="white", alpha=0.3)

plt.xlabel("Similarity betw. generated and true light sequence")
plt.ylabel("Frequency")

# Save before showing the plot
plt.savefig("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2gpt/plots/histplot_bins_40_poly_kde_lightskyblue.png", dpi=300)
plt.show()

In [ ]:
df[!, :heavy_sep_light] = string.(df.input_heavy_sequence,"[SEP]", df.sequence_alignment_aa_light,)

# Display the first few rows to verify
println(first(df, 10))

In [ ]:
names(df)

In [ ]:
# Step 1: Count occurrences of each value in the column
value_counts = combine(groupby(df, :heavy_sep_light), nrow => :count)

# Step 2: Filter for values that appear more than once
duplicates = filter(row -> row.count > 1, value_counts)

# Display duplicate entries
println("Duplicate entries in heavy_sep_light:")
println(duplicates)


In [ ]:

# Test DataFrame: df_main
df_main = DataFrame(
    sequence_alignment_heavy_sep_light = [
        "SEQ1 [SEP] LIGHT1",
        "SEQ2 [SEP] LIGHT2",
        "SEQ3 [SEP] LIGHT3",
        "SEQ3 [SEP] LIGHT3",
        "SEQ4 [SEP] LIGHT4",
        "SEQ5 [SEP] LIGHT5"
    ],
    BType = ["Plasma", "Memory", "Naive", "Naive", "Memory", "Plasma"],
    Disease = ["None", "Flu", "None", "None", "Covid", "None"]
)

# Test DataFrame: df_other
df_other = DataFrame(
    heavy_sep_light = [
        "SEQ1 [SEP] LIGHT1",
        "SEQ2 [SEP] LIGHT2",
        "SEQ2 [SEP] LIGHT2",  # Duplicate
        "SEQ4 [SEP] LIGHT4",
        "SEQ6 [SEP] LIGHT6"   # Not in df_main
    ],
    BLOSUM_score = [90, 85, 85, 88, 92],
    similarity = [0.98, 0.95, 0.95, 0.89, 0.93]
)

println("df_main:")
println(df_main)
println("df_other:")
println(df_other)


In [ ]:
# Step 1: Remove duplicates from the "other" DataFrame
df_main_unique = unique(df_main, :sequence_alignment_heavy_sep_light)

# Step 2: Merge the DataFrames using `on`
df_merged = innerjoin(
    df_main_unique, 
    df_other, 
    on=(:sequence_alignment_heavy_sep_light => :heavy_sep_light)
)

println("\nMerged DataFrame:")
println(df_merged)

In [ ]:
# /ibmm_data2/oas_database/paired_lea_tmp/paired_model/coherence_analysis_in_oas_db/data/full_extraction_for_coherence_paired_data_extra_cols_header.csv -> extracted paired data with B Types
df_main_full = CSV.read("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/coherence_analysis_in_oas_db/data/full_extraction_for_coherence_paired_data_extra_cols_header.csv", DataFrame);

In [ ]:
# Step 1: Remove duplicates from the "other" DataFrame
df_main_full_unique = unique(df_main_full, :sequence_alignment_heavy_sep_light)

In [ ]:
names(df_main_full_unique)

In [ ]:

# Extract IDs from query file
function get_ids(file)
    ## Populate set with lines from the file
    queries = Set(eachline(file))
    return queries
end

# Modified compare_ids function to accept a file handle for output -> IDs here: heavy[SEP]light sequences!
function compare_ids(queries, db, output_file)
    # Loop through db_lines
    for ln in eachline(db)
        # Extract ID with regex
        parts = split(ln, ',')  # Splitting based on comma
        #ID = strip(parts[end])  # Assuming the ID is the last part after the comma!
        ID = strip(parts[15])  # Extracting ID from the xth column!! -> always check the column number in your dataset!
        # Search in queries
        if ID in queries
            println(output_file, ln)  # Writing to file instead of standard output
        end
    end
end

In [ ]:

# Extract the heavy_sep_light column as a new DataFrame
df_ids = select(df, :heavy_sep_light)

# Save the extracted column to a CSV file
output_file_path = "/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2bert/data/heavy_sep_light_column_ids.csv"
CSV.write(output_file_path, df_ids)

println("Column saved to $output_file_path")

In [ ]:
# # Open output file for writing
# open("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2bert/data/extracted_data_merged.csv", "w") do output_file
#     # Run the actual functions with output redirection
#     open("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2bert/data/heavy_sep_light_column_ids.csv") do query_file
#         queries = get_ids(query_file)
#         open("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/coherence_analysis_in_oas_db/data/full_extraction_for_coherence_paired_data_extra_cols_header.csv") do db_file
#             compare_ids(queries, db_file, output_file)
#         end
#     end
# end

In [ ]:
# Function to get the column names from a CSV file
function get_column_names(file)
    first_line = readline(file)  # Read the first line from the file
    columns = split(first_line, ",")  # Split by commas to get individual column names
    return columns
end

In [ ]:


# Open output file for writing
open("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2bert/data/extracted_data_merged_header.csv", "w") do output_file
    # Open query file and extract queries
    open("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2bert/data/heavy_sep_light_column_ids.csv") do query_file
        queries = get_ids(query_file)

        # Open db_file and extract column names
        open("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/coherence_analysis_in_oas_db/data/full_extraction_for_coherence_paired_data_extra_cols_header.csv") do db_file
            # Get column names from db_file
            column_names = get_column_names(db_file)

            # Write the column names to the output file
            println(output_file, join(column_names, ","))  # Write header

            # Now we can run the compare_ids function with output redirection
            # Reopen the db_file as it is already consumed by get_column_names
            seekstart(db_file)  # Go back to the beginning of the file
            compare_ids(queries, db_file, output_file)
        end
    end
end

In [ ]:
extracted_data_merged = CSV.read("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2bert/data/extracted_data_merged_header.csv", DataFrame)


In [ ]:
df_merged_final = innerjoin(extracted_data_merged, df, on=(:sequence_alignment_heavy_sep_light => :heavy_sep_light), makeunique=true)

In [ ]:
@df df_merged_final histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=false
)


In [ ]:

@df df_merged_final histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=true,  # Show the legend
    group=:BType  # Group by the BType column to color the histogram bars
)

In [ ]:
df_filtered_mem_naive = filter(row -> row.BType in ["Memory-B-Cells", "Naive-B-Cells"], df_merged_final)

In [ ]:
@df df_filtered_mem_naive histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=true,  # Show the legend
    group=:BType  # Group by the BType column to color the histogram bars
)

In [ ]:
@df df_merged_final histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=true,  # Show the legend
    group=:locus_light  # Group by the BType column to color the histogram bars
)

In [ ]:


@df df_merged_final histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=true,  # Show the legend
    group=:Author  # Group by the BType column to color the histogram bars
)

In [ ]:
@df df_merged_final histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=true,  # Show the legend
    group=:Subject  # Group by the BType column to color the histogram bars
)

In [ ]:
# Extract up to the first two segments (e.g., "IGKV2-30" from "IGKV2-30*01")
df_merged_final[!, :general_v_gene_heavy] = replace.(df_merged_final.v_call_heavy, r"(^[^*]+?)(?:\*.*)?$" => s"\1"); # the regex removes any part of the string after (and including) the first *, keeping only the portion before it.
df_merged_final[!, :general_v_gene_light] = replace.(df_merged_final.v_call_light, r"(^[^*]+?)(?:\*.*)?$" => s"\1");

In [ ]:
@df df_merged_final histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=true,  # Show the legend
    group=:general_v_gene_heavy  # Group by the BType column to color the histogram bars
)

In [ ]:
@df df_merged_final histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=true,  # Show the legend
    group=:general_v_gene_light  # Group by the BType column to color the histogram bars
)

In [ ]:

# Extract up to the first two segments (e.g., "IGKV2-30" from "IGKV2-30*01")
df_merged_final[!, :v_gene_heavy_family] = replace.(df_merged_final.v_call_heavy, r"(^[^*]+?)(?:\-.*)?$" => s"\1"); # the regex removes any part of the string after (and including) the first *, keeping only the portion before it.
df_merged_final[!, :v_gene_light_family] = replace.(df_merged_final.v_call_light, r"(^[^*]+?)(?:\-.*)?$" => s"\1");

df_merged_final;


In [ ]:
@df df_merged_final histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=true,  # Show the legend
    group=:v_gene_heavy_family  # Group by the BType column to color the histogram bars
)

In [ ]:
@df df_merged_final histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=true,  # Show the legend
    group=:v_gene_light_family  # Group by the BType column to color the histogram bars
)


In [ ]:
# Group by the column v_gene_heavy_family
grouped_df = groupby(df_merged_final, :v_gene_light_family)

# Calculate the average and group size for each group
result_df = combine(grouped_df, 
    :calculated_similarity => mean => :average_similarity,  # Calculate average similarity
    nrow => :group_size                                      # Calculate group size
)

# Display the resulting DataFrame
println(result_df)

In [ ]:

# Group by the column v_gene_light_family
grouped_df = groupby(df_merged_final, :v_gene_light_family)

# Calculate the average, group size, and number of unique values in generated_light_seq for each group
result_df = combine(grouped_df, 
    :calculated_similarity => mean => :average_similarity,  # Calculate average similarity
    nrow => :group_size,                                    # Calculate group size
    :generated_sequence_light => x -> length(unique(x)) => :num_unique_generated_light_seq  # Count unique values
)

# Display the resulting DataFrame
result_df

In [ ]:
# sort v_gene_light_family by its most common entry
sorted_df = sort(result_df, :group_size, rev=true)

In [ ]:
# Group by the column BType
grouped_df = groupby(df_merged_final, :BType)

# Calculate the average and group size for each group
result_df = combine(grouped_df, 
    :calculated_similarity => mean => :average_similarity,  # Calculate average similarity
    nrow => :group_size                                      # Calculate group size
)

# Display the resulting DataFrame
println(result_df)

In [ ]:
# Group by the column BType
grouped_df = groupby(df_merged_final, :v_gene_heavy_family)

# Calculate the average similarity for each group
result_df = combine(grouped_df, 
    :calculated_similarity => mean => :average_similarity,
    nrow => :group_size   # Calculate the average of calculated_similarity
)

# Display the resulting DataFrame
println(result_df)

In [ ]:
# Group by the column BType
grouped_df = groupby(df_merged_final, :locus_light)

# Calculate the average similarity for each group
result_df = combine(grouped_df, 
    :calculated_similarity => mean => :average_similarity,
    nrow => :group_size   # Calculate the average of calculated_similarity
)

# Display the resulting DataFrame
println(result_df)

In [ ]:
df_filtered_kappa_only = filter(row -> row.locus_light in ["K"], df_merged_final);




In [ ]:
@df df_filtered_kappa_only histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=true,  # Show the legend
    group=:v_gene_light_family  # Group by the BType column to color the histogram bars
)

In [ ]:
@df df_filtered_kappa_only histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=true,  # Show the legend
    group=:v_gene_heavy_family  # Group by the BType column to color the histogram bars
)

In [ ]:
grouped_df = groupby(df_filtered_kappa_only, :v_gene_light_family)

# Calculate the average similarity for each group
result_df = combine(grouped_df, 
    :calculated_similarity => mean => :average_similarity,
    nrow => :group_size   # Calculate the average of calculated_similarity
)

# Display the resulting DataFrame
println(result_df)

In [ ]:
grouped_df = groupby(df_merged_final, :general_v_gene_light)

# Calculate the average similarity for each group
result_df = combine(grouped_df, 
    :calculated_similarity => mean => :average_similarity,
    nrow => :group_size   # Calculate the average of calculated_similarity
)

# Display the resulting DataFrame
println(result_df)

In [ ]:
# save df_merged_final to a CSV file
output_file_path = "/ibmm_data2/oas_database/paired_lea_tmp/paired_model/coherence_analysis_in_oas_db/data/test_set/df_merged_final_test_set.csv"
CSV.write(output_file_path, df_merged_final)



In [ ]:


# Group by the column BType
grouped_df = groupby(df_merged_final, :germline_alignment_aa_light)

grouped_df_filt = filter(g -> nrow(g) > 10, grouped_df);

# Calculate the average similarity for each group
result_df = combine(grouped_df_filt, 
    :calculated_similarity => mean => :average_similarity,
    nrow => :group_size
)

# Sort the DataFrame by average_similarity in descending order
sorted_result_df = sort(result_df, :average_similarity, rev=true)

# Display the resulting sorted DataFrame
sorted_result_df

In [ ]:
function score_bio(S1, S2; matrix=BLOSUM62, gop=-10, gep=-4)
	scoremodel = AffineGapScoreModel(matrix, gap_open=gop, gap_extend=gep)
	res = pairalign(GlobalAlignment(), S1, S2, scoremodel)
	res
end

score_bio("DVVVTQSPLSLPVTLGQPASISCGSSESLLYSNGNTYLSWFQQRPGQSPRRLIYQVSNRDSGVPDRFSGSGSGTDFTLKISRVEVEDVGVYFCMQGTYLPITFGQGTRLEIK", "DVVMTQSPLSLPVTLGQPASISCRSSQSLVYSDGNTYLNWFQQRPGQSPRRLIYKVSNRDSGVPDRFSGSGSGTDFTLKISRVEAEDVGVYYCMQGTHWPITFGQGTRLEIK")

In [ ]:

function score_bio(S1, S2; matrix=BLOSUM62, gop=-10, gep=-4)
    # Create the scoring model
    scoremodel = AffineGapScoreModel(matrix, gap_open=gop, gap_extend=gep)
    
    # Perform the alignment
    res = pairalign(GlobalAlignment(), S1, S2, scoremodel)

    aln = alignment(res)
    
    matches = count_matches(aln)
    
    # Calculate the percentage similarity
    alignment_length = length(S1)  # Includes gaps
    similarity_percentage = (matches / alignment_length) * 100
    
    return similarity_percentage
end

# Example usage
seq1 = aa"DIQMTQSPSSLSASIGDRVTITCRASQDINNYLAWYQQKPGKVPKLLIYAASTLHSGVPSRFSGSGSGTDFSLTISSLQPDDIASYYCQKYNSAITFGQGTRLDIK"
seq2 = aa"DIQMTQSPSSLSASVGDRVTITCRASQGISNYLAWYQQKPGKVPKLLIYAASTLQSGVPSRFSGSGSGTDFTLTISSLQPEDVATYYCQKYNSAITFGQGTRLEIK"

similarity = score_bio(seq1, seq2)

println("Similarity Percentage: ", similarity, "%")
similarity

test = "x"
print("$x")

In [ ]:
# Define the score_bio function
function score_bio(S1, S2; matrix=BLOSUM62, gop=-10, gep=-4)
    # Create the scoring model
    scoremodel = AffineGapScoreModel(matrix, gap_open=gop, gap_extend=gep)
    
    # Perform the alignment
    res = pairalign(GlobalAlignment(), S1, S2, scoremodel)

    # Extract alignment and count matches
    aln = alignment(res)
    matches = count_matches(aln)
    
    # Calculate the percentage similarity
    alignment_length = length(S1)  # Includes gaps
    similarity_percentage = (matches / alignment_length) * 100
    
    return similarity_percentage
end

# Ensure the relevant columns are strings
df_merged_final.sequence_alignment_aa_light = string.(df_merged_final.sequence_alignment_aa_light)
df_merged_final.germline_alignment_aa_light = string.(df_merged_final.germline_alignment_aa_light)

# Calculate alignment similarity for each row
df_merged_final.alignment_germline = [
    score_bio(row.sequence_alignment_aa_light, row.germline_alignment_aa_light)
    for row in eachrow(df_merged_final)
]


In [ ]:
# using DataFrames

# # Add a new column for the similarity group
# df_merged_final.group = [
#     if 0 ≤ val < 55
#         "0-55"
#     elseif 55 ≤ val < 75
#         "55-75"
#     elseif 75 ≤ val ≤ 100
#         "75-100"
#     else
#         "Out of range"  # Optional, to catch invalid values
#     end for val in df_merged_final.calculated_similarity
# ]

# # Group the DataFrame by the new column
# grouped_df = groupby(df_merged_final, :group)

# # Display grouped data
# for g in grouped_df
#     println("Group: ", g.group[1])
#     println(g)
# end

In [ ]:
using Pkg
Pkg.add("CategoricalArrays")

In [ ]:
using DataFrames
using CategoricalArrays

# Define the similarity bins
bins = [0, 55, 75, 110]

# Create a new column by binning the calculated_similarity values
df_merged_final.group = cut(df_merged_final.calculated_similarity, bins, labels=["0-55", "55-75", "75-100"])

# Group the DataFrame by the new column
grouped_df = groupby(df_merged_final, :group)

# Display grouped data
for g in grouped_df
    println("Group: ", g.group[1])
    println(g)
end

In [ ]:
# Group the DataFrame by the 'group' column and calculate the mean of 'calculated_similarity' for each group
mean_df = combine(grouped_df, :alignment_germline => mean)

# Display the resulting DataFrame
mean_df

In [ ]:
@df df_merged_final histogram(
    :calculated_similarity, 
    bins=150,  # Number of bins
    xlabel="Calculated Similarity", 
    ylabel="Frequency", 
    title="Distribution of Calculated Similarity", 
    legend=true,  # Show the legend
    group=:Subject  # Group by the BType column to color the histogram bars
)

In [ ]:
# save df_merged_final to a CSV file
output_file_path = "/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2gpt/data/bert2gpt_df_merged_final_test_set.csv"
CSV.write(output_file_path, df_merged_final)